# Hallucination Project

Let's start by gathering all libraries and modules needed:

In [76]:
from load_data import *
import json
from pathlib import Path
import glob
import os
import numpy as np

We start by extracting the Abstract and Reasoning Corpus (ARC) dataset.
This dataset is divided by training and evaluation

In [56]:
data_path = Path(
    "/Users/mattiapiazza/Documents/University/cognitive_behavioral_and_social_data/project/cbsd_hallucination_project/ARC_master/data"
)

training_dataset = data_extraction(data_path=Path(data_path / "training"))
# training_dataset = data_extraction(data_path=data_path / "training")
evaluation_dataset = data_extraction(data_path=data_path / "evaluation")

# Small check
print(f"""training_dataset: {len(training_dataset["train"]["input"])}
evaluation_dataset: {len(evaluation_dataset["train"]["input"])}""")

training_dataset: 1301
evaluation_dataset: 1363


In [87]:
for key, value in training_dataset.items():
    for k, val in value.items():
        for v in val:
            print(f"Our matrix:\n{from_num_to_col(v)}\n")

Our matrix:
1 -> XXG
2 -> XGX
3 -> GXX


Our matrix:
1 -> XXG
2 -> XXG
3 -> XXG


Our matrix:
1 -> GXX
2 -> XGX
3 -> GXX


Our matrix:
1 -> XGX
2 -> XXG
3 -> XGX


Our matrix:
1 -> VVVVVVVYVYV
2 -> VVVVVVVVVVV
3 -> VVVVBVVVVVV
4 -> VVVVVVVVVVV
5 -> VVBVVVBYVYV
6 -> VVVVVVVVVVV
7 -> VVVVBVRVVVR
8 -> VVVVVVVVVVV
9 -> CVVVCVVVVVV
10 -> VVVVVVVVVVV
11 -> CVVVCVRVVVR
12 -> VVVVVVVVVVV


Our matrix:
1 -> BBBBBBBB
2 -> BCBCBBBB
3 -> BBBBBBBB
4 -> BCBCBBBB
5 -> BBBBBBBB
6 -> BBBBVBBB
7 -> BBBVBVBB
8 -> BBBBVBBB
9 -> BBBBBBBB
10 -> BBBBBBBB


Our matrix:
1 -> YYYYYYYYYYYYYY
2 -> YYBYYYBYYYYYYY
3 -> YYYYYYYYYYYYYY
4 -> YYYYYYYYYYYYYY
5 -> YYYYYYYYYYYYYY
6 -> YYBYYYBYYOYYYY
7 -> YYYYYYYYYYYYYY
8 -> YYYYYYYOYYYOYY
9 -> YYYYYYYYYYYYYY
10 -> YYYYYYYYYOYYYY
11 -> YYYYYYYYYYYYYY
12 -> YYYYYYYYYYYYYY


Our matrix:
1 -> XXXXXXXXXXXXXXX
2 -> XXXXXXXXXXXXXXX
3 -> RRRRRRRRRRRRRRR
4 -> XXXXXXXXXXXXXXX
5 -> XXXXXXXXXXXXXXX
6 -> BBBBBBBBBBBBBBB
7 -> XXXXXXXXXXXXXXX
8 -> XXXXXXXXXXXXXXX
9 -> VVVVVVVVVVVVVVV
10

### Question our LLM
Now that we have our prompts it's time to use them to question the LLM chosen